In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import glob
import random
from PIL import Image
import cv2
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D,Dense,Flatten,Dropout,AvgPool2D
from keras.layers.normalization import BatchNormalization
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from keras.applications import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
print(os.listdir("../input"))

In [ ]:
def make_directory(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

In [ ]:
import zipfile


DATASET_PATH = './dogs-vs-cats'

make_directory(DATASET_PATH)

# extract train data
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall(DATASET_PATH)

# extract test data
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall(DATASET_PATH)

In [ ]:
TRAIN_DATA_PATH = os.path.sep.join([DATASET_PATH, 'train'])
TEST_DATA_PATH = os.path.sep.join([DATASET_PATH, 'test1'])
train_files = os.listdir(TRAIN_DATA_PATH)
test_files = os.listdir(TEST_DATA_PATH)

print(len(train_files), len(test_files))

In [ ]:
Image.open(os.path.sep.join([TRAIN_DATA_PATH, train_files[0]])) 


In [ ]:
print(os.path.sep.join([TRAIN_DATA_PATH, train_files[0]])) 

In [ ]:
train_files

In [ ]:
make_directory('./dogs-vs-cats/new_train')
for i in train_files:
    dir_path = i.split('/')[-1]
    if dir_path.split(".")[0] not in os.listdir('./dogs-vs-cats/new_train/'):
        make_directory('./dogs-vs-cats/new_train/' + dir_path.split(".")[0])
    shutil.copy('./dogs-vs-cats/train/' + i  ,'./dogs-vs-cats/new_train/' + dir_path.split(".")[0] + '/' + i)    

In [ ]:
print(os.listdir('./dogs-vs-cats/new_train/dog/'))

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator 

image_size = 224
image_path = './dogs-vs-cats/new_train'
train_datagen=ImageDataGenerator(
                        rescale=1./255,
                        validation_split=0.1,
                        horizontal_flip=False,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        rotation_range=40,
                        fill_mode='nearest'
                        )

train_generator=train_datagen.flow_from_directory(
                        image_path, 
                        target_size=(image_size,image_size),
                        subset='training',
                        shuffle=True,
                        batch_size=128,
                        class_mode='categorical'
                        )

valid_datagen=ImageDataGenerator(
                        validation_split=0.1,
                        rescale=1./255
                        )

valid_generator=valid_datagen.flow_from_directory(
                        image_path, 
                        target_size=(image_size,image_size),
                        subset='validation',
                        shuffle=False,
                        batch_size=128,
                        class_mode='categorical'
                        )

In [ ]:
x,y = train_generator.next()
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
for i in range(0,10):
    image = x[i]
    plt.imshow(image)
    c=0
    for i in y[i]:
        if i==0:
            c+=1
        else:break
    label=labels[c]
    label=label.replace('_',' ')
    label=label.lower()
    plt.title(label)
    plt.axis('off')
    plt.show()

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

Inception = InceptionV3(input_shape = (224,224,3), 
                                 include_top = False, 
                                 weights = 'imagenet')


model=Sequential()
model.add(Inception)

In [ ]:
model.summary()

In [ ]:
model.add(GlobalAveragePooling2D())
model.add(Dense(512 , activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(512 , activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(2 , activation = 'softmax'))
model.summary()

In [ ]:
model.compile(optimizer = 'adam' ,
    loss = 'categorical_crossentropy' ,
              metrics = ['accuracy']
             )

In [ ]:
callback=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5,min_delta=0,mode='auto',restore_best_weights=False,baseline=None)

history=model.fit(train_generator,
                   steps_per_epoch=None,
                   epochs=4,
                   validation_data=valid_generator,
                   validation_steps=4,
                   callbacks=[callback])

In [ ]:
os.mkdir('./model/')
model.save('./model/dog_cat.h5')

In [ ]:
def plotModel(history):
    fig,(ax1 , ax2) = plt.subplots(1 , 2 , figsize = (12 , 4))
    fig.suptitle('Train vs Val')
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.title.set_text('Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train' , 'val'] , loc = 4)
    
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.title.set_text('Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['Train' , 'Val'] , loc = 1 )
    
    fig.show()
    
plotModel(history)    

In [ ]:
test_filenames = os.listdir("./dogs-vs-cats/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./dogs-vs-cats/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    batch_size=32,
    target_size=(224, 224),
    shuffle=False
)

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/32))

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
test_df

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
sample_test = test_df.head(18)
sample_test.head()

In [ ]:
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filenam = row['filename']
    category = row['category']
    img = load_img("./dogs-vs-cats/test1/"+str(filenam))
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)